In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#https://scipython.com/blog/quadtrees-2-implementation-in-python/
#https://pydoc.net/openslide-python/1.1.1/openslide/

import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

In [3]:
## HACK get rid of it
import sys
sys.path.append("..")
from registration_tree import Rect, QuadTree

In [4]:
source_slide_paths = list(Path(r"D:\Datasets\ScannerStudy\Aperio\CCMCT").glob("*.svs"))

len(source_slide_paths)

5

In [5]:
target_slide_paths = list(Path(r"D:\Datasets\ScannerStudy\AxioScan\CCMCT").glob("*.tif")) 

len(target_slide_paths)

5

In [6]:
source_slide_paths = {name.stem.split("_")[-2] : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in source_slide_paths}
source_slide_paths.keys()

dict_keys(['183715A', '22108', '29609B', '380609B', '518711B'])

In [7]:
target_slide_paths = {name.stem.split("_")[-2] : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in target_slide_paths}
target_slide_paths.keys()

dict_keys(['183715A', '22108', '29609B', '380609B', '518711B'])

In [8]:
for name in source_slide_paths:
    if name not in target_slide_paths:
        print(name)

In [9]:
for name in target_slide_paths:
    if name not in source_slide_paths:
        print(name)

In [10]:
df = pd.read_csv("CCMCT.csv", delimiter=";", names=["image_name", "grade", "vector"])
df.head()

,image_name,grade,vector
0,A_CCMCT_29609B_1.svs,mitosis,"{""x1"": 57204,""y1"": 28229,""x2"": 57254,""y2"": 28279}"
1,A_CCMCT_29609B_1.svs,mitosis,"{""x1"": 57895,""y1"": 37440,""x2"": 57945,""y2"": 37490}"
2,A_CCMCT_29609B_1.svs,mitosis,"{""x1"": 54810,""y1"": 37131,""x2"": 54860,""y2"": 37181}"
3,A_CCMCT_29609B_1.svs,mitosis,"{""x1"": 54575,""y1"": 34537,""x2"": 54625,""y2"": 34587}"
4,A_CCMCT_29609B_1.svs,mitosis,"{""x1"": 47883,""y1"": 28677,""x2"": 47933,""y2"": 28727}"


In [11]:
df = pd.read_csv("CCMCT.csv", delimiter=";", names=["image_name", "grade", "vector"])
df["vector"] = [json.loads(vector)  for vector in df["vector"]]

df["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in df["vector"]]

df["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in df["vector"]]

df["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(df["x1"], df["x2"])]
df["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(df["y1"], df["y2"])]

df["anno_width"] = [x2-x1 for x1, x2 in zip(df["x1"], df["x2"])]
df["anno_height"]= [y2-y1 for y1, y2 in zip(df["y1"], df["y2"])]

df.head()

,image_name,grade,vector,x1,y1,x2,y2,center_x,center_y,anno_width,anno_height
0,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 57204, 'y1': 28229, 'x2': 57254, 'y2': ...",57204,28229,57254,28279,57229.0,28254.0,50,50
1,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 57895, 'y1': 37440, 'x2': 57945, 'y2': ...",57895,37440,57945,37490,57920.0,37465.0,50,50
2,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 54810, 'y1': 37131, 'x2': 54860, 'y2': ...",54810,37131,54860,37181,54835.0,37156.0,50,50
3,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 54575, 'y1': 34537, 'x2': 54625, 'y2': ...",54575,34537,54625,34587,54600.0,34562.0,50,50
4,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 47883, 'y1': 28677, 'x2': 47933, 'y2': ...",47883,28677,47933,28727,47908.0,28702.0,50,50


In [12]:
%%time

results = {}
target_depth = 2

for name, source in source_slide_paths.items():
    
    target = target_slide_paths[name]
      
        
    source_slide = source["slide"]
    target_slide = target["slide"]
    
    source_dimension = Rect.create(Rect, 0, 0, source_slide.dimensions[0], source_slide.dimensions[1])
    target_dimension = Rect.create(Rect, 0, 0, target_slide.dimensions[0], target_slide.dimensions[1])
    
    qtree = QuadTree(source_dimension, source_slide, target_dimension, target_slide, target_depth=target_depth, debug=True)
    
    results[source["name"]] = qtree

error: OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\core\src\batch_distance.cpp:275: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == CV_32F || type == CV_8U) in function 'cv::batchDistance'


In [13]:
result_folder = Path(f"Depth_{target_depth}")

result_folder.mkdir(parents=True, exist_ok=True)

In [14]:
import pickle

for path, qtree in results.items():
    
    with open(result_folder / f'{qtree.source_name}-To-{qtree.target_name}.pickle', 'wb') as handle:
        pickle.dump(qtree, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
for path, qtree in results.items():
    
    df_gt = df[df["image_name"] == path.name]#.sample(n=5, random_state=1)
    
    annos = np.array(df_gt[["center_x", "center_y", "anno_width", "anno_height"]]) 
    qtree.draw_annotations(annos, num_sub_pic=5)